In [ ]:
import pandas as pd
import numpy as np
from nilearn import datasets
from nilearn import maskers
import matplotlib.pyplot as plt

from nilearn import plotting
from nilearn.connectome import ConnectivityMeasure

import networkx as nx
from scipy.spatial.distance import cdist

In [ ]:
#Load ADHD file 
adhd_filename = pd.read_csv("E:/ADHD200/path_label_motion_spatial.csv")
adhd_filename

In [ ]:
adhd_pos=[]
adhd_neg=[]

for i in range(len(adhd_filename)):
    if(adhd_filename['Label'][i]==0):
        adhd_neg.append(adhd_filename['motion_spatial'][i])
    else:
        adhd_pos.append(adhd_filename['motion_spatial'][i])

In [ ]:
data_dir = "E:/ADHD200/"

# Dictionary learning  component decide to generate ROI

In [ ]:
from nilearn.image import resample_img
from nilearn.masking import compute_multi_epi_mask
from nilearn.decomposition import DictLearning

# Resample all images to a common FOV
reference_img = resample_img(adhd_motion_spatial[0], target_affine=None, target_shape=None)
reference_img_shape = reference_img.shape[:3]  # get the first three dimensions
reference_img_affine = reference_img.affine

adhd_resampled = []
for img in adhd_motion_spatial[0:100]:
    img_resampled = resample_img(img, target_affine=reference_img.affine, target_shape=reference_img_shape)
    adhd_resampled.append(img_resampled)

# Compute a common mask for all resampled images
mask_img = compute_multi_epi_mask(adhd_resampled, threshold=0.3)

# Fit the ICA model using the resampled images and the common mask
dict_learning= DictLearning(
    n_components=20,
    memory="nilearn_cache",
    memory_level=2,
    verbose=1,
    random_state=0,
    n_epochs=1,
    mask_strategy="whole-brain-template",
    mask=mask_img
)

dict_learning.fit(adhd_resampled)

In [ ]:
dictlearning_components_img = dict_learning.components_img_
# components_img is a Nifti Image object, and can be saved to a file with
# the following line:
dictlearning_components_img.to_filename('E:/ADHD200/dictlearning_components_img.nii.gz')

In [ ]:
import numpy as np
import nibabel as nib

# load the .nii.gz file
dictlearning_components_img = nib.load('E:/ADHD200/dictlearning_components_img.nii.gz')

# get the data as a numpy array
dictlearning_components_numpy= dictlearning_components_img.get_fdata()


# Print all dictionary components

In [ ]:
from nilearn.plotting import plot_prob_atlas
# Plot all DL components together
plot_prob_atlas(
    dictlearning_components_img, title="All DictLearning components"
)

# Generate region extrator for DL

In [ ]:
from nilearn.regions import RegionExtractor

extractor = RegionExtractor(
    dictlearning_components_img,
    threshold=0.5,
    thresholding_strategy="ratio_n_voxels",
    extractor="local_regions",
    standardize=True,
    min_region_size=1350,
)
# Just call fit() to process for regions extraction
extractor.fit()
# Extracted regions are stored in regions_img_
regions_extracted_img = extractor.regions_img_
# Each region index is stored in index_
regions_index = extractor.index_
# Total number of regions extracted
n_regions_extracted = regions_extracted_img.shape[-1]

# Visualization of region extraction results
# title = (
#     "%d regions are extracted from %d components."
#     "\nEach separate color of region indicates extracted region"
#     % (n_regions_extracted, 8)

plotting.plot_prob_atlas(
    regions_extracted_img, view_type="filled_contours"
)

# Display all atoms or components

In [ ]:
from nilearn.image import iter_img
from nilearn.plotting import plot_stat_map, show

for i, cur_img in enumerate(iter_img(dictlearning_components_img)):
    plot_stat_map(
        cur_img,
        display_mode="z",
        title=f"Comp {int(i)}",
        cut_coords=1,
        colorbar=False,
    )

# Time series data generation

In [ ]:
from nilearn.connectome import ConnectivityMeasure
all_data=[]
for filename  in adhd_motion_spatial:
    time_series = extractor.transform(filename)
    all_data.append(time_series) 

In [ ]:
np.save("E:/ADHD200/dict_learning_times_series_905.npy",all_data)